In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import asyncio
import nest_asyncio
import os
import pandas as pd
from sqlalchemy import inspect
from sqlmodel import SQLModel, create_engine, Session, select
from urllib.parse import quote

from property_scraper.pvp.models.sale import Attachment

from pypdl.downloader import AsyncFileDownloader

In [3]:
nest_asyncio.apply()

In [4]:
database_url = "sqlite:////media/emanuele/data/property_scraper/demos/pvp.sqlite"
table = 'attachments'
column = 'attachment_link'
rootname = 'https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it'
output_folder : str = '/media/emanuele/data/property_scraper/demos/downloads/pvp/attachments'

In [5]:
engine = create_engine(database_url)

In [6]:
SQLModel.metadata.create_all(engine)

In [7]:
def get_columns_inspect(model_or_table):
    return [column.name for column in inspect(model_or_table).columns]

In [8]:
get_columns_inspect(Attachment)

['id',
 'attachment_id',
 'file_name',
 'attachment_link',
 'size',
 'description',
 'attachment_type_code',
 'is_lot_insertion',
 'content_base64',
 'asset_id']

In [9]:
with Session(engine) as session:
    statement = select(Attachment.attachment_link)
    results = session.exec(statement).all()
    urls = [os.path.join(rootname, quote(result[1:], safe=':/?=&')) for result in results]
    print(len(urls))
    print('\n'.join(urls[:3]))
df = pd.DataFrame(urls)
df.to_csv('/media/emanuele/data/property_scraper/demos/downloads/pvp/attachments/urls.csv', index=False)

518245
https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/99095/Perizia.pdf?versionId=adc6bd71-e423-4fd3-ada3-a7385369ad2b
https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/99095/Avviso%20di%20vendita.pdf?versionId=80993e26-9bf8-4e6f-8915-f06730f2024a
https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/99095/Ordinanza%20vendita.pdf?versionId=8e98d584-b327-45e3-b0b8-1ca2f917ba18


In [10]:
print(len(urls))

518245


In [11]:
urls[0]

'https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/99095/Perizia.pdf?versionId=adc6bd71-e423-4fd3-ada3-a7385369ad2b'

In [15]:
async def main():
    async with AsyncFileDownloader() as downloader:
        result = await downloader.download_file(urls[0], output_folder=output_folder)
        print(f"Content: {content[:10]}")

In [19]:
async def main():
    async with AsyncFileDownloader() as downloader:
        results = await downloader.download_all(urls[:10000], output_folder=output_folder)

In [20]:
asyncio.run(main())


Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [03:11<00:00, 19.17s/file]


KeyboardInterrupt: 